In [ ]:
from ros_edf.ros_interface import EdfRosInterface
from edf.pc_utils import pcd_from_numpy, draw_geometry, reconstruct_surface
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, save_demos

import torch
import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

### Initialize EDF ROS Interface

In [ ]:
env_interface = EdfRosInterface(reference_frame = "scene")

### Initialize dataset list

In [ ]:
dataset = []

### Observe Point clouds

In [ ]:
grasp_pc = env_interface.observe_eef(obs_type = 'pointcloud', update = True)
scene_pc = env_interface.observe_scene(obs_type = 'pointcloud', update = True)

### Pick

In [ ]:
target_poses = SE3([0.0, 0.0, 1.0, 0.0, -0.05, 0.0, 0.275])
result = env_interface.pick(target_poses=target_poses)

pick_success = result[-1] is not None

In [ ]:
if True: # pick_success:
    pick_demo = TargetPoseDemo(target_poses=target_poses, scene_pc=scene_pc, grasp_pc=grasp_pc)

### Move to observe gripper

In [ ]:
grasp_pc = env_interface.observe_eef(obs_type = 'pointcloud', update = True)
env_interface.attach(pcd = grasp_pc)

In [ ]:
env_interface.move_to_target_pose(poses = SE3([0.0, 0.0, 1.0, 0.0, 0.00, 0.0, 0.6]))

### Observe gripper

In [ ]:
env_interface.detach()
grasp_pc = env_interface.observe_eef(obs_type = 'pointcloud', update = True)
env_interface.attach(pcd = grasp_pc)

### Observe Scene

In [ ]:
env_interface.move_to_target_pose(SE3([0.0, 0.0, 1.0, 0.0, -0.4, 0.4, 0.6]))

In [ ]:
scene_pc = env_interface.observe_scene(obs_type = 'pointcloud', update = True)

### Place

In [ ]:
target_poses = SE3([0.0, 0.0, 1.0, 0.0, -0.05, 0.0, 0.35])
pre_grasp_results, grasp_pose = env_interface.move_to_target_pose(poses = target_poses)

In [ ]:
env_interface.detach()
env_interface.release()

In [ ]:
place_demo = TargetPoseDemo(target_poses=target_poses, scene_pc=scene_pc, grasp_pc=grasp_pc)

### Save

In [ ]:
demo_seq = DemoSequence(demo_seq = [pick_demo, place_demo])
dataset.append(demo_seq)
save_demos(demos=dataset, dir="demo/test_demo")